Python Threat Hunting _ Command Line Arguments

Command line arguments are parameters or options that are passed to a program or script when it is executed via the command line. They let the user of the script modify behavior or configure its settings, typically by providing inputs (e.g. files, strings, the output of another command) or flags (e.g. -h for help). If an input is provided via command line arguments then it is parsed by the script into a usable format and then used somewhere within the script.

Command line arguments are often used to automate repetitive tasks, configure programs, or to pass inputs that are too complex to be entered via a GUI. They are also used to build more complex scripts and programs that can be executed with a variety of options and inputs, making them more versatile and powerful.

In [13]:
import requests, csv
from bs4 import BeautifulSoup
import argparse, json
import sys

# filter out jupyter kernel arguments for error handling
filtered_args = [arg for arg in sys.argv[1:] if not arg.startswith('--f=')]

# define commandline argument
parser = argparse.ArgumentParser()
parser.add_argument('--csv', action='store_true', help='output in CSV format')
parser.add_argument('--json', action='store_true', help='output in JSON format')
parser.add_argument('--txt', action='store_true', help='output in text format')
parser.add_argument('--url', type=str, default="https://www.cisa.gov/news-events/bulletins/sb25-209", help='URL of CISA bulletins to scrape')
args = parser.parse_args(filtered_args)

# define output file table header
Header_row = ["Product", "Vendor", "Description", "Published", "CVSS", "CVE", "Reference"]

# define output functions for csv, json, txt
def output_csv(vulns):
    csv_file = Filename + ".csv"
    with open(csv_file, "w", encoding='UTF8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(Header_row)

        for vuln in vulns:
            data_row = [vuln['product'], vuln['vendor'], vuln['description'], vuln['published'], vuln['cvss'], vuln['cve'], vuln['reference']]
            writer.writerow(data_row)

def output_json(vulns):
    json_file = Filename + ".json"
    with open(json_file, "w", encoding='UTF8') as file:
        json.dump(vulns, file, indent=2)

def output_txt(vulns):
    txt_file = Filename + ".txt"
    with open(txt_file, "w", encoding='UTF8') as file:
        file.write(" |\t".join(Header_row) + "\n")
        file.write("-" * 100 + "\n")

        for vuln in vulns:
            data = vuln['product'] + " | " + vuln['vendor'] + " | " + vuln['description'] + " | " + vuln['published'] + " | " + vuln['cvss'] + " | " + vuln['cve'] + " | " + vuln['reference']
            file.write(data + "\n")

# download CISA page
try:
    page = requests.get(args.url)
    page.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")
    sys.exit(1)

# parse CISA page with BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

# define output
Page_Title = soup.title.string
a = soup.title.string.split("of")
b = a[1].split("|")
Filename = "CISA vulnerabilities - " + b[0].strip()

# capture high vulnerabilities table
table = soup.find("table")
table_body = table.find("tbody")
rows = table_body.find_all("tr")

vulns = []

# for loop to extract relevant fields from the table
for row in rows:
    cols = [x for x in row.find_all("td")]

    product, vendor = cols[0].text.split("--")
    description = cols[1].text.strip()
    published = cols[2].text.strip()
    cvss = cols[3].text.strip()
    cve = cols[4].find("a").text.strip()
    reference = cols[4].find("a").get("href")

    vuln = {
        "product": product.strip(),
        "vendor": vendor.strip(),
        "description": description,
        "published": published,
        "cvss": cvss,
        "cve": cve,
        "reference": reference
    }

    vulns.append(vuln)  # append dictionary objects to vulns list

# define output format
if args.csv:
    output_csv(vulns)
elif args.json:
    output_json(vulns)
elif args.txt:
    output_txt(vulns)
else:
    print(vulns)

print(f"Processed {len(vulns)} vulnerabilities from: {Page_Title}")
print(f"Files saved with prefix: {Filename}")

[{'product': '1000 Projects', 'vendor': 'ABC Courier Management System', 'description': 'A vulnerability has been found in 1000 Projects ABC Courier Management System 1.0 and classified as critical. Affected by this vulnerability is an unknown functionality of the file /Add_reciver.php. The manipulation of the argument reciver_name leads to sql injection. The attack can be launched remotely. The exploit has been disclosed to the public and may be used.', 'published': '2025-07-25', 'cvss': '7.3', 'cve': 'CVE-2025-8173', 'reference': 'https://www.cve.org/CVERecord?id=CVE-2025-8173'}, {'product': '1000 Projects', 'vendor': 'ABC Courier Management System', 'description': 'A vulnerability was found in 1000 Projects ABC Courier Management System 1.0. It has been classified as critical. Affected is an unknown function of the file /getbyid.php. The manipulation of the argument ID leads to sql injection. It is possible to launch the attack remotely. The exploit has been disclosed to the public 